In [10]:
import os
import argparse
from pathlib import Path
from multiprocessing import Pool, cpu_count

import numpy as np
from tqdm import tqdm

from package_bond_search_algorithm.algolithm_bond_search_for_tetrahedral_shape import concat_filter
from package_bond_search_algorithm.package_file_conversion.nnlist2df import nnlist2df

In [2]:
central_atom_symbol = 'N'
neighboring_atom_symbol = 'H' 
bond_length_lower_end = 0.82
bond_length_upper_end = 1.24
# ターゲットとなるイオンの元素種を含むPOSCARのディレクトリパス一覧を取得
target_npy_p = '../get_some_speceis_existed_poscar_path_list/N_H_existed_poscar_folder_path_list.npy'
some_species_existed_poscar_folder_path_list = np.load(target_npy_p, allow_pickle=True)
print(f"len(some_species_existed_poscar_folder_path_list): {len(some_species_existed_poscar_folder_path_list)}")
some_species_existed_poscar_folder_path_list = some_species_existed_poscar_folder_path_list[0:100]

In [3]:
def nnlist2df_and_concat_filter(nnlist_path, central_atom_symbol, neighboring_atom_symbol, bond_length_lower_end, bond_length_upper_end):
    df_nnlist = nnlist2df(nnlist_path=nnlist_path)
    return_bool = concat_filter(df_nnlist=df_nnlist, 
                                central_atom_symbol=central_atom_symbol,
                                neighboring_atom_symbol=neighboring_atom_symbol,
                                bond_length_lower_end=bond_length_lower_end,
                                bond_length_upper_end=bond_length_upper_end)
    return return_bool


def wrap_nnlist2df_and_concat_filter(args):
    return nnlist2df_and_concat_filter(*args)


def mk_job_args(some_species_existed_nnlist_folder_p_list, central_atom_symbol, neighboring_atom_symbol, bond_length_lower_end, bond_length_upper_end):
    # ターゲットとなるイオンの元素種を含むPOSCAR.nnlistのディレクトリパス一覧を取得
    add_str = '/nnlist_5/POSCAR.nnlist'
    some_species_existed_nnlist_p_list = [str(p) + add_str for p in some_species_existed_nnlist_folder_p_list]
    number_of_nnlist = len(some_species_existed_nnlist_folder_p_list)
    central_atom_symbol_list = [central_atom_symbol for i in range(number_of_nnlist)]
    neighboring_atom_symbol_list = [neighboring_atom_symbol for i in range(number_of_nnlist)]
    bond_length_lower_end_list = [bond_length_lower_end for i in range(number_of_nnlist)]
    bond_length_upper_end_list = [bond_length_upper_end for i in range(number_of_nnlist)]
    job_args = zip(some_species_existed_nnlist_p_list,
                   central_atom_symbol_list,
                   neighboring_atom_symbol_list,
                   bond_length_lower_end_list,
                   bond_length_upper_end_list)
    return job_args    

In [5]:
job_args = mk_job_args(some_species_existed_poscar_folder_path_list,
                       central_atom_symbol=central_atom_symbol,
                       neighboring_atom_symbol=neighboring_atom_symbol,
                       bond_length_lower_end=bond_length_lower_end,
                       bond_length_upper_end=bond_length_upper_end)

In [7]:
# 並列化
pp = Pool(cpu_count() - 1)
total = len(some_species_existed_poscar_folder_path_list)
try:
    bool_ion_contain = list(tqdm(pp.imap(wrap_nnlist2df_and_concat_filter, job_args), total=total))
finally:
    pp.close()
    pp.join()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 119.66it/s]


In [9]:
# apply filter(:bool_ion_contain) to N_H_existed_poscar_nnlist_path_list
ion_contained_poscar_nnlist_path_list = np.array(some_species_existed_poscar_folder_path_list)[bool_ion_contain]
# get ion contained folder path list
ion_contained_poscar_folder_path_list = [Path(os.path.split(Path(os.path.split(Path(p))[0]))[0]) for p in ion_contained_poscar_nnlist_path_list]
# save ion contained folder path list as .npy
saved_npy_fname = 'NH4_contained_poscar_folder_path_list_ver2.npy'
np.save(saved_npy_fname, ion_contained_poscar_folder_path_list)
# print parcentage
print(f"len(ion_contained_poscar_folder_path_list)/len(N_H_existed_poscar_folder_path_list) :\
{len(ion_contained_poscar_folder_path_list)}/{total}")

len(ion_contained_poscar_folder_path_list)/len(N_H_existed_poscar_folder_path_list) :11/100


In [ ]:
# def wrap_nnlist2df_and_concat_filter(nnlist_p):
#     """
#     This func() executes two func()s.: nnlistdf() and concat_filter().

#     Usage:
#     ------
#     wrap_nnlist2df_and_concat_filter(nnlist_p=nnlist_p)

#     Parameter:
#     -----------
#     nnlist_p: str or PosixPath

#     Return:
#     -------
#     bool: True or False
#     """
#     df_nnlist = nnlist2df(nnlist_p)
#     bool_ = concat_filter(df_nnlist=df_nnlist)

#     return bool_


# ターゲットとなるイオンの元素種を含むPOSCARのディレクトリパス一覧を取得
target_npy_p = '../get_some_speceis_existed_poscar_path_list/N_H_existed_poscar_folder_path_list.npy'
N_H_existed_poscar_folder_path_list = np.load(target_npy_p, allow_pickle=True)
print(f"len(N_H_existed_poscar_folder_path_list): {len(N_H_existed_poscar_folder_path_list)}")
N_H_existed_poscar_folder_path_list = N_H_existed_poscar_folder_path_list

# ターゲットとなるイオンの元素種を含むPOSCAR.nnlistのディレクトリパス一覧を取得
add_str = '/nnlist_5/POSCAR.nnlist'
N_H_existed_poscar_nnlist_path_list = [str(p) + add_str for p in N_H_existed_poscar_folder_path_list]

# 並列化
pp = Pool(cpu_count() - 1)
total = len(N_H_existed_poscar_nnlist_path_list)
try:
    bool_ion_contain = list(tqdm(pp.imap(wrap_nnlist2df_and_concat_filter, N_H_existed_poscar_nnlist_path_list), total=total))
finally:
    pp.close()
    pp.join()

# apply filter(:bool_ion_contain) to N_H_existed_poscar_nnlist_path_list
ion_contained_poscar_nnlist_path_list = np.array(N_H_existed_poscar_nnlist_path_list)[bool_ion_contain]
# get ion contained folder path list
ion_contained_poscar_folder_path_list = [Path(os.path.split(Path(os.path.split(Path(p))[0]))[0]) for p in ion_contained_poscar_nnlist_path_list]
# save ion contained folder path list as .npy
saved_npy_fname = 'NH4_contained_poscar_folder_path_list.npy'
np.save(saved_npy_fname, ion_contained_poscar_folder_path_list)
# print parcentage
print(f"len(ion_contained_poscar_folder_path_list)/len(N_H_existed_poscar_folder_path_list) :\
{len(ion_contained_poscar_folder_path_list)}/{total}")
